## Code for Creating SUMMA Forcing Files
* Taken from, at some point, Bart's code for creating SUMMA forcing files / ncdfs into the correct format 

In [1]:
# import packages 
# %matplotlib widget
%matplotlib inline

# plotting packages 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 

# interactive plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots # adding for subplots
import plotly.figure_factory as ff

# data packages 
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime

import csv 
import copy 
import os.path 

In [31]:
ds =  pd.read_csv("/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/Sodankyla/SOD_1819_1hr_cleaned.csv")
# sod['datetime'] = pd.to_datetime(sod['datetime'])
ds.index = pd.DatetimeIndex(ds['datetime'], name='time')
# sod.drop(columns=['datetime'], inplace=True)

ds.head()

,datetime,shortwave,longwave,ppt_1,ppt1,RH,windspeed,pressure,temp
time,,,,,,,,,
2018-10-01 00:00:00,2018-10-01 00:00:00,0.0,327.45,0.0,0.0,98.5,2.64420,97135.0,3.15
2018-10-01 01:00:00,2018-10-01 01:00:00,0.0,321.75,0.0,0.0,97.5,2.23740,97185.0,2.45
2018-10-01 02:00:00,2018-10-01 02:00:00,0.0,316.05,0.0,0.0,96.5,1.83060,97235.0,1.75
2018-10-01 03:00:00,2018-10-01 03:00:00,0.0,312.60,0.0,0.0,96.0,1.72890,97295.0,1.10
2018-10-01 04:00:00,2018-10-01 04:00:00,0.0,311.20,0.0,0.0,96.5,1.67805,97360.0,0.65


In [34]:
ds = ds.loc['2018-10-01T00:00:00':'2019-05-11T23:00:00'] # doing this for now because SUMMA is crashing at May 12 so I want to see if this is a quick fix (not a good fix) 

In [35]:
ds['precip'] = ds['ppt_1'].copy(deep=True)
ds['temp_K'] = ds['temp'] + 273.15
ds['pressure_Pa'] = ds['pressure'] / 100

In [36]:
# Calculating the specific humidtiy from RH, temperature, and precip
def spechum(rh, T, p):
    T0 = 273.15
    return rh * np.exp((17.67*(T-T0))/(T-29.65)) / (0.263*p)

In [37]:
sod_spechum = spechum(ds.RH, ds.temp_K, ds.pressure_Pa)

In [38]:
shortwave = ds.shortwave
longwave = ds.longwave
precip = ds.precip
pressure = ds.pressure_Pa
airtemp = ds.temp_K
windspd = ds.windspeed
spechum = sod_spechum
timestamp = ds.index

In [39]:
# USING BARTS OLD CODE FOR CREATING 

# Attributes in the netCDF file, with their units and the full length name of the variable 
attrs = {
   'airpres':  {'units': 'Pa', 'long_name': 'Air pressure'},
   'airtemp':  {'units': 'K', 'long_name': 'Air temperature'},
   'spechum':  {'units': 'g g-1', 'long_name': 'Specific humidity'},
   'windspd':  {'units': 'Wind speed', 'long_name': 'm s-1'},
   'SWRadAtm': {'units': 'W m-2', 'long_name': 'Downward shortwave radiation'},
   'LWRadAtm': {'units': 'W m-2', 'long_name': 'Downward longwave radiation'},
   'pptrate':  {'units': 'kg m-2 s-1', 'long_name': 'Precipitation rate'}}

#mpl.rcParams['figure.figsize'] = (18, 10)

#______________________________________________________________________________________________

# bounds = pd.DatetimeIndex(pd.date_range('2016/10/01', '2017/05/17', freq='H'), name='time')[1:-1]
# ds['index'] = pd.DatetimeIndex(ds['datetime'], name='time')
# ds['index'] = pd.DatetimeIndex(timestamp, name='time')

# Need to change the latitude, longitude, and elevation
lats = [67.361800] 
lons = [26.638167]
elev = 180

bounds = ds.index

shape = (len(bounds), 1, )
dims = ('time', 'hru', )
coords = {'time': bounds}

# Time stepping
met_data = xr.Dataset(coords=coords)
met_data.time.encoding['calendar'] = 'standard'
met_data.time.encoding['units'] = 'hours since 1990-01-01'

# The variables in SUMMA that are in the forcings
summa_vars = ['airpres', 'airtemp', 'spechum', 
              'windspd', 'SWRadAtm', 'LWRadAtm', 'pptrate']
for varname in summa_vars:
    met_data[varname] = xr.DataArray(data=np.full(shape, np.nan),
                                     coords=coords, dims=dims,
                                     name=varname, attrs=attrs[varname])

In [40]:
met_data['airpres' ].loc[{'hru': 0}] = pressure      #air pressure [Pa]
met_data['airtemp' ].loc[{'hru': 0}] = airtemp       #temperature [K]
met_data['windspd' ].loc[{'hru': 0}] = windspd       #wind speed [m s-1]
met_data['SWRadAtm'].loc[{'hru': 0}] = shortwave     #shortwave [W m-2]
met_data['LWRadAtm'].loc[{'hru': 0}] = longwave      #longwave [w m-2]
met_data['pptrate' ].loc[{'hru': 0}] = precip        #precip [kg m-2 s-1]
met_data['spechum' ].loc[{'hru': 0}] = spechum       #specific humidity [g/g-1]

# Using the attributes from the local_attributes netCDF instead of recreating them for the forcings 

ds_local_attrs = xr.open_dataset('/Users/Lumbr/OneDrive - UW/Documents/Washington/Lumbrazo_et_al_2022\Modeling_Files/params/local_attributes.nc')
#ds_template = xr.open_dataset('./CUES/cues_2016_summa_setup/forcings/cues2016.nc')

# lsosmet = xr.open_dataset('/storage/lumbraca/SnowEx2017_metdata/Niwot_2016_fixed_final-Copy1.nc')
# ds_local_attrs = lsosmet

met_data['hruId'] = ds_local_attrs['hruId']
met_data['latitude'] = ds_local_attrs['latitude']
met_data['longitude'] = ds_local_attrs['longitude']
met_data['data_step'] = [3600.]

met_data.to_netcdf('/Users/Lumbr/OneDrive - UW/Documents/Washington/UnloadingRegimes/OtherSites/Sodankyla/Sodankyla_SUMMAForcing_FromBart_cutdates.nc')
met_data

<xarray.Dataset>
Dimensions:    (time: 5352, hru: 1, data_step: 1)
Coordinates:
  * time       (time) datetime64[ns] 2018-10-01 ... 2019-05-11T23:00:00
  * data_step  (data_step) float64 3.6e+03
Dimensions without coordinates: hru
Data variables:
    airpres    (time, hru) float64 971.4 971.9 972.4 973.0 ... 993.9 993.6 993.0
    airtemp    (time, hru) float64 276.3 275.6 274.9 274.2 ... 280.8 280.5 280.4
    spechum    (time, hru) float64 0.4832 0.4549 0.4281 ... 0.6291 0.6229 0.617
    windspd    (time, hru) float64 2.644 2.237 1.831 1.729 ... 1.627 1.678 2.288
    SWRadAtm   (time, hru) float64 0.0 0.0 0.0 0.0 0.0 ... 24.2 8.2 2.3 0.35 0.0
    LWRadAtm   (time, hru) float64 327.4 321.8 316.1 312.6 ... 348.2 344.8 328.9
    pptrate    (time, hru) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    hruId      (hru) int32 ...
    latitude   (hru) float64 ...
    longitude  (hru) float64 ...

In [41]:
met_data.time.tail

<bound method DataArray.tail of <xarray.DataArray 'time' (time: 5352)>
array(['2018-10-01T00:00:00.000000000', '2018-10-01T01:00:00.000000000',
       '2018-10-01T02:00:00.000000000', ..., '2019-05-11T21:00:00.000000000',
       '2019-05-11T22:00:00.000000000', '2019-05-11T23:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2018-10-01 ... 2019-05-11T23:00:00>